# 載入使用套件

In [1]:
!pip install tomorrow

  Created wheel for tomorrow: filename=tomorrow-0.2.4-py3-none-any.whl size=3129 sha256=1f17a29d3f0ebc9e3950109e7b194429157379e0658800f0a0177c9cd01ff04d
  Stored in directory: /Users/Rebeca/Library/Caches/pip/wheels/b1/f8/c7/c42b1c10a31468f3062e2bf2f8d2c0748e9cd983293b96a9bc
Successfully built tomorrow


In [1]:
# 爬蟲類套件
import requests
import json
from bs4 import BeautifulSoup
# 加入使用者資訊(如使用什麼瀏覽器、作業系統...等資訊)模擬真實瀏覽網頁的情況
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'}
from tomorrow import threads
# 資料整理類套件
import pandas as pd
import re
from datetime import datetime

# 其他
import os

# 擷取一篇新聞

In [2]:
def GetNews_chinatimes(response):
    soup = BeautifulSoup(response.text)
    url = soup.find('link')['href']
    time=datetime.strptime(soup.find('meta', attrs={'property':'article:published_time'})['content'],'%Y-%m-%dT%H:%M:%S+08:00')
    ndf = pd.DataFrame(data = [{
                                #'TITLE':soup.find('h1', attrs={'class':'article-title'}).text,
                                'TIME':time,
                                #'CATEGORY':soup.find('meta',attrs={'property':'article:section'})['content'],
                                #'DESCRIPTION':soup.find('meta',attrs={'name':'description'})['content'],
                                'CONTENT':'\n'.join(i.text for i in soup.find('div',attrs={'class':'article-body'}).find_all('p')),
                                #'KEYWORDS':soup.find('meta',{'name':'news_keywords'})['content'],
                                #'FROM':soup.find('meta',{'name':'publisher'})['content']
                                'LINK':soup.find('meta', {'property':'og:url'})['content']}],
                       columns = [#'TITLE', 
                                  'TIME', 
                           #'CATEGORY', 'DESCRIPTION', 
                           'CONTENT'])
                           #,'KEYWORDS', 'FROM', 'LINK']) 
    return ndf

In [13]:
url = 'https://www.chinatimes.com/newspapers/20191028000424-260118?chdtv'
resp = requests.get(url)
df3 = GetNews_chinatimes(resp)

In [11]:
df2['TIME'][0].month#.split('-')[1]

10

In [16]:
df5=pd.DataFrame()

# 擷取特定關鍵詞新聞的連結清單

In [3]:
def GetLinks_chinatimes(response):
    links = []
    soup = BeautifulSoup(response.text)
    for i in soup.find_all('h3'):
        url = i.find('a')['href']
        links.append(url)
    return links

In [0]:
url = 'https://www.chinatimes.com/search/美國'
resp = requests.get(url)
GetLinks_chinatimes(resp)

['https://www.chinatimes.com/newspapers/20200131000218-260203',
 'https://www.chinatimes.com/realtimenews/20200131001427-260408',
 'https://www.chinatimes.com/realtimenews/20200131000791-260408',
 'https://www.chinatimes.com/newspapers/20200131000370-260208',
 'https://www.chinatimes.com/newspapers/20200131000217-260203',
 'https://www.chinatimes.com/newspapers/20200131000375-260208',
 'https://www.chinatimes.com/newspapers/20200131000200-260203',
 'https://www.chinatimes.com/newspapers/20200131000117-260301',
 'https://www.chinatimes.com/realtimenews/20200131003148-260410',
 'https://www.chinatimes.com/realtimenews/20200131002544-260410',
 'https://www.chinatimes.com/realtimenews/20200131001828-260407',
 'https://www.chinatimes.com/hottopic/20200131003838-260804',
 'https://www.chinatimes.com/opinion/20200131000004-262115',
 'https://www.chinatimes.com/realtimenews/20200131000771-260408',
 'https://www.chinatimes.com/realtimenews/20200131001564-260408',
 'https://www.chinatimes.com/ho

## 股票關鍵字清單

In [8]:
symbol=[]
stock=[]
ss=[]
f = open(r'/Users/Rebeca/Fintech_中信專案/成交金額前20大.txt')

stock_num=[]
for line in f:
    s=re.findall(r"\d+",line) #\d 匹配任意数字
    name = line.split("        ")[1]
    name = name.split("\n")[0]
    #print(line)
    #print(name)
    ss.append(name)
    for i in s:
         ss.append(i)



# for i in f:
#     num = i.split('\t')[0]
#     name= re.sub(r'\n', "", i.split('\t')[1])
#     symbol.append(num)
#     stock.append(name)
#     ss.append(num)
#     ss.append(name)
f.close
print(ss)

['台積電', '2330', '聯發科', '2454', '世芯-KY', '3661', '穩懋', '3105', '愛普', '6531', '立積', '4968', '義隆', '2458', '旺宏', '2337', '環球晶', '6488', '矽力-KY', '6415', '南亞科', '2408', '譜瑞-KY', '4966', '精測', '6510', '瑞昱', '2379', '信驊', '5274', '祥碩', '5269', '神盾', '6462', '聯電', '2303', '嘉晶', '3016', '宏捷科', '8086']


In [5]:
print(len(symbol))
print(len(stock))
print(len(ss))

152
152
304


## 開啟多線程功能

In [9]:
@threads(5)
def MultiThread_Crawl(url):
    try:
        return requests.get(url, headers=headers)
    except:
        pass

# 組合應用

In [10]:
def CrawlNews_chinatimes(keywords, pages):
    # 截取多個分頁的新聞連結
    links = []
    for i in range(pages):
        url = 'https://www.chinatimes.com/search/{}?page={}'.format(keywords, i+1)
        resp = requests.get(url)
        links += GetLinks_chinatimes(resp)
        links = list(set(links))  
        print('There are {} links in page {}.'.format(len(links),str(i+1)))

    # 多線程爬蟲
    responses = [MultiThread_Crawl(link) for link in links]

    # 整理成DataFrame
    list_of_dataframes = []
    for response in responses:
        try:
            ndf = GetNews_chinatimes(response)
            if (ndf['TIME'][0].year==2020)&(ndf['TIME'][0].month >4):
                list_of_dataframes.append(ndf)
            else:
                break
        except:
            pass
    df = pd.concat(list_of_dataframes, ignore_index=True)
    print('There are {} News in DataFrame.'.format(len(df)))
    return df

- 爬蟲前可以先手動查詢大約有多少新聞

In [44]:
%%time
# 可以自行替換查詢的關鍵字，另外需要更多新聞的人也可以把 pages 的數值調高)
df = CrawlNews_chinatimes(keywords='台積電', pages=25)

There are 20 links in page 0.
There are 40 links in page 1.
There are 60 links in page 2.
There are 80 links in page 3.
There are 100 links in page 4.
There are 120 links in page 5.
There are 140 links in page 6.
There are 160 links in page 7.
There are 180 links in page 8.
There are 200 links in page 9.
There are 200 News in DataFrame.
CPU times: user 6.52 s, sys: 480 ms, total: 7 s
Wall time: 13.6 s


In [56]:
len(df)

200

## 最後執行

In [11]:
df_all=pd.DataFrame()
for key in ss:
    try:
        df = CrawlNews_chinatimes(key, pages=25)
        df_all = df_all.append(df)
        
    except:
        continue
df_all.reset_index(drop=True,inplace=True)

There are 20 links in page 1.
There are 40 links in page 2.
There are 60 links in page 3.
There are 80 links in page 4.
There are 100 links in page 5.
There are 120 links in page 6.
There are 140 links in page 7.
There are 160 links in page 8.
There are 180 links in page 9.
There are 200 links in page 10.
There are 220 links in page 11.
There are 240 links in page 12.
There are 260 links in page 13.
There are 280 links in page 14.
There are 300 links in page 15.
There are 320 links in page 16.
There are 340 links in page 17.
There are 360 links in page 18.
There are 380 links in page 19.
There are 400 links in page 20.
There are 420 links in page 21.
There are 440 links in page 22.
There are 460 links in page 23.
There are 480 links in page 24.
There are 500 links in page 25.
There are 500 News in DataFrame.
There are 0 links in page 1.
There are 0 links in page 2.
There are 0 links in page 3.
There are 0 links in page 4.
There are 0 links in page 5.
There are 0 links in page 6.
There 

There are 360 links in page 18.
There are 380 links in page 19.
There are 400 links in page 20.
There are 420 links in page 21.
There are 440 links in page 22.
There are 460 links in page 23.
There are 480 links in page 24.
There are 500 links in page 25.
There are 0 links in page 1.
There are 0 links in page 2.
There are 0 links in page 3.
There are 0 links in page 4.
There are 0 links in page 5.
There are 0 links in page 6.
There are 0 links in page 7.
There are 0 links in page 8.
There are 0 links in page 9.
There are 0 links in page 10.
There are 0 links in page 11.
There are 0 links in page 12.
There are 0 links in page 13.
There are 0 links in page 14.
There are 0 links in page 15.
There are 0 links in page 16.
There are 0 links in page 17.
There are 0 links in page 18.
There are 0 links in page 19.
There are 0 links in page 20.
There are 0 links in page 21.
There are 0 links in page 22.
There are 0 links in page 23.
There are 0 links in page 24.
There are 0 links in page 25.
The

There are 1 links in page 10.
There are 1 links in page 11.
There are 1 links in page 12.
There are 1 links in page 13.
There are 1 links in page 14.
There are 1 links in page 15.
There are 1 links in page 16.
There are 1 links in page 17.
There are 1 links in page 18.
There are 1 links in page 19.
There are 1 links in page 20.
There are 1 links in page 21.
There are 1 links in page 22.
There are 1 links in page 23.
There are 1 links in page 24.
There are 1 links in page 25.
There are 20 links in page 1.
There are 40 links in page 2.
There are 60 links in page 3.
There are 80 links in page 4.
There are 100 links in page 5.
There are 120 links in page 6.
There are 140 links in page 7.
There are 160 links in page 8.
There are 174 links in page 9.
There are 174 links in page 10.
There are 174 links in page 11.
There are 174 links in page 12.
There are 174 links in page 13.
There are 174 links in page 14.
There are 174 links in page 15.
There are 174 links in page 16.
There are 174 links i

There are 80 links in page 4.
There are 100 links in page 5.
There are 120 links in page 6.
There are 140 links in page 7.
There are 160 links in page 8.
There are 180 links in page 9.
There are 200 links in page 10.
There are 220 links in page 11.
There are 240 links in page 12.
There are 260 links in page 13.
There are 280 links in page 14.
There are 300 links in page 15.
There are 320 links in page 16.
There are 340 links in page 17.
There are 360 links in page 18.
There are 380 links in page 19.
There are 400 links in page 20.
There are 420 links in page 21.
There are 440 links in page 22.
There are 460 links in page 23.
There are 480 links in page 24.
There are 500 links in page 25.
There are 0 links in page 1.
There are 0 links in page 2.
There are 0 links in page 3.
There are 0 links in page 4.
There are 0 links in page 5.
There are 0 links in page 6.
There are 0 links in page 7.
There are 0 links in page 8.
There are 0 links in page 9.
There are 0 links in page 10.
There are 0 

In [12]:
df_all

,TIME,CONTENT
0,2020-05-19 14:30:12,台積電周二爆量多、空大鬥法，股價卻僅在291.5元至294元狹幅震盪，顯示市場人氣、法人著墨...
1,2020-06-03 14:13:51,在全球短期利率多創歷史新低，與新冠疫情趨緩，各主要工業國重啟經濟活動愈趨審慎樂觀下，使國際熱...
2,2020-05-13 04:10:00,外資12日再翻臉賣超台股集中市場253.66億元，累計5月來賣超擴大至831億元，今年來已賣...
3,2020-06-02 04:10:00,傳統產業出頭天，證交所1日公布，108年由上緯投控以516萬元，奪下平均員工薪資（不含福利費...
4,2020-05-26 04:10:00,上市櫃年度現金股利大致出爐，受疫情打亂企業成長腳步之下，整體配發金額1.36兆元，法人認為，...
5,2020-05-14 11:42:26,川普政府擔憂美國過度依賴亞洲半導體供應鏈，點名台積電、英特爾等大廠赴美設廠。但知名半導體分析...
6,2020-06-06 23:56:00,華為受川普政府新禁令制裁，可能面臨被台積電斷貨危機，但對台積電來說，若失去第二大客戶華為，今...
7,2020-05-28 15:23:08,美股由黑翻紅，四大指數齊收漲，亞股也跟上走揚，台股周三開漲25.96點至11040.62點，...
8,2020-05-17 15:00:00,一隻病毒，打亂了所有的節奏。可是，能將危機化為轉機的企業，會在最低谷時，大膽出手搶人；戰役過...
9,2020-06-02 08:12:48,小編精選5件不可不知財經大事，帶讀者掌握今天（2日）財經重點。\n\n【1】中國反制 暫停美...


In [14]:
len(df_all)

505

In [45]:
df_all['CONTENT'][1324]

'全球工業級SSD儲存解決方案領導商宇瞻科技(8271)，於八月六日至九日在美國Santa Clara的2018 Flash Memory Summit展出全新工規3D固態硬碟儲存方案和儲存技術，滿足IoT世代高容量、高速、高耐受度的儲存需求。\n\n宇瞻今年營運重點在維持穩健獲利，不再像去年衝刺營收及市占率；7月營收8.07億元，月減0.18%，較去年同月減少12.38%。1-7月營收57.13億元，較2017年同期增加4.08%。公司將於10日下午在證交所場地辦理法人說明會。\n\n\n\n萬物互連時代來臨，從雲端運算所需的大型資料中心至近期崛起的邊緣運算，資料中心持續建置，智慧裝置亦呈現爆炸性成長，高容量儲存設備方能滿足全球急遽增加的資料量；而在IoT世代下，儲存設備數量增加並座落於各地，固態硬碟的儲存行為管理將成為重要課題；處於IoT應用邊緣的智慧裝置，以隨機小資料讀寫行為居多，其儲存產品耐受度將面臨新考驗。\n\n為滿足海量資料的儲存需求，宇瞻科技於2018 Flash Memory Summit展示全新正工規3D固態硬碟儲存方案，以及CoreLife和CoreAnalyzer2儲存技術和管理工具，不僅產品儲存容量倍增，亦大幅提升產品耐受度和管理便利性，提供用戶靈活且具成本優勢的儲存解決方案。\n\n宇瞻科技推出全新3D正工規儲存解決方案，採用3D TLC NAND顆粒，各式固態硬碟規格皆具備高容量選擇；搭載NVMe介面和Over-Provisioning技術，連續讀寫速度高達3300/2600 MB/sec，加上耐寬溫規格，全系列產品可廣泛地運用於交通、軍事、博弈、保健等各領域IoT應用。\n\n\n\n\n(時報資訊)'

# 儲存

In [15]:
df_all.to_csv('/Users/Rebeca/Fintech_中信專案/中時data_0609.csv')

# 儲存結果

## 串接 Google Drive 空間

In [75]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

## 保存資料
- 資料會保存在 Google Drive 上 Colab Notebooks/WebCrawler/TW_News/News 的資料夾空間中

In [0]:
import os
path = '/content/drive/My Drive/Colab Notebooks/WebCrawler/TW_News/News'
# 如果不存在這個資料夾就新建這個資料夾
if not os.path.exists(path):
    os.makedirs(path)
df.to_excel(path + '/News_chinatimes.xlsx')